# Install packages and configure GCP

In [ ]:
%pip install requests

In [ ]:
%pip install --upgrade google-cloud-aiplatform

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud config set project "your-project-goes-here"

In [ ]:
!gcloud auth application-default login

In [ ]:
!gcloud init

# Install and set up dkg.py, Python SDK for interacting with the DKG

In [ ]:
%pip install --upgrade dkg
%pip install python-dotenv
%pip show dkg

In [ ]:
from dkg import DKG
from dkg.providers import BlockchainProvider, NodeHTTPProvider
from dotenv import load_dotenv
import os
import json

dotenv_path = './.env'
load_dotenv(dotenv_path)
jwt_token = os.getenv('JWT_TOKEN')
ot_node_hostname = os.getenv('OT_NODE_HOSTNAME')
private_key = os.getenv('PRIVATE_KEY')

node_provider = NodeHTTPProvider(ot_node_hostname, jwt_token)
blockchain_provider = BlockchainProvider("mainnet", "otp:2043", private_key=private_key, gas_price=40)

dkg = DKG(node_provider, blockchain_provider)
print(dkg.node.info)

# Instruct Gemini to create a Knowledge Asset on the DKG

In [ ]:
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Content, Part, GenerationConfig

def get_chat_response(chat: ChatSession, prompt: str):
  response = chat.send_message(prompt, generation_config=GenerationConfig(temperature=0))
  return response.candidates[0].content.parts[0].text


instruction_message = '''
Construct a JSON object following the artwork JSON-LD schema based on the provided information by the user.
The user will provide the image URL, artwork name, description, artform, author name. For the publisher name you can assume 'Google Demo Amsterdam'.
Add 5 keywords based on the artwork name and description.
Use the provided image URL for the id field as well.

Here's an example of an artwork that corresponds to the mentioned JSON-LD schema:
{
  "@context": "http://schema.org",
  "@type": "VisualArtwork",
  "@id": "https://origintrail.io/images/otworld/1fc7cb79f299ee4.jpg",
  "name": "The Last Supper",
  "description": "The Last Supper is an iconic Renaissance fresco by Leonardo Da Vinci.",
  "artform": "Painting",
  "author": { "@type": "Person", "name": "Leonardo da Vinci" },
  "image": "https://origintrail.io/images/otworld/1fc7cb79f299ee4.jpg",
  "keywords": [ "The Last Supper", "Leonardo da Vinci", "Renaissance", "fresco", "religious art" ],
  "publisher": { "@type": "Person", "name": "dkgbrka" }
}

Output the JSON as a string, between ```json and ```.
'''

instruction_understood_message = "Yes."


chat_history = [
    Content(parts=[Part.from_text(instruction_message)], role="user"),
    Content(parts=[Part.from_text(instruction_understood_message)], role="model"),
]

In [ ]:
gemini_pro_model = GenerativeModel("gemini-1.0-pro-001", generation_config=GenerationConfig(temperature=0))
chat = gemini_pro_model.start_chat(history=chat_history)
question = '''Create an artwork with name: Decentralized Hexagon, created by OriginTrail. Description should be "Digital art showcasing the OriginTrail network in an artistic way" and you can conclude the artform yourself. Here's the image URL https://i.gyazo.com/72b6cd16e0d5b2e131b0311456dcdefc.png'''
generated_json = get_chat_response(chat, question)

print(generated_json)

In [ ]:
def clean_json_string(input_string):
    if input_string.startswith("```json") and input_string.endswith("```"):
        cleaned_query = input_string[7:-3].strip()
        return cleaned_query
    elif input_string.startswith("```") and input_string.endswith("```"):
        cleaned_query = input_string[3:-3].strip()
    else:
        return input_string

In [ ]:
artwork = json.loads(clean_json_string(generated_json))

content = {"public": artwork}
create_asset_result = dkg.asset.create(content, 2)
print('Asset created!')
print(json.dumps(create_asset_result, indent=4))
print(create_asset_result["UAL"])

get_asset_result = dkg.asset.get(create_asset_result["UAL"])
print(json.dumps(get_asset_result, indent=4))

# Instruct Gemini to construct a SparQL query in order to query the DKG

In [ ]:
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Content, Part, GenerationConfig

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    response = chat.send_message(prompt, generation_config=GenerationConfig(temperature=0))
    print(response)

    return response.candidates[0].content.parts[0].text

instruction_message = '''
I am working on a project involving artworks and their related data. I have a schema in JSON-LD format that outlines the structure and relationships of the data I am dealing with. Based on this schema, I need to construct a SPARQL query to retrieve specific information from a dataset that follows this schema.

The schema is focused on artworks and includes various properties such as the artist, description, artform and author among others. My goal with the SPARQL queries is to retrieve data from the graph about the artworks, based on the natural language question that the user posed.

Here's an example of an artwork the JSON-LD format: { "@context": "http://schema.org", "@type": "VisualArtwork", "@id": "https://origintrail.io/images/otworld/1fc7cb79f299ee4.jpg", "name": "The Last Supper", "description": "The Last Supper is an iconic Renaissance fresco by Leonardo Da Vinci.", "artform": "Painting", "author": { "@type": "Person", "name": "Leonardo da Vinci" }, "image": "https://origintrail.io/images/otworld/1fc7cb79f299ee4.jpg", "keywords": [ "The Last Supper", "Leonardo da Vinci", "Renaissance", "fresco", "religious art" ], "publisher": { "@type": "Person", "name": "dkgbrka" } }

Here's an example of a query to find artworks from publisher "BranaRakic":
```sparql
PREFIX schema: <http://schema.org/>

SELECT ?artwork ?name ?ual

WHERE { ?artwork a schema:VisualArtwork ;
GRAPH ?g
{ ?artwork schema:publisher/schema:name "BranaRakic" ; schema:name ?name . }

?ual schema:assertion ?g
FILTER(CONTAINS(str(?ual), "2043")) }```

Pay attention to retrieving the UAL, this is a mandatory step of all your queries. After getting the artwork with '?artwork a schema:VisualArtwork ;' you should wrap the next conditions around GRAPH ?g { }, and later use the graph retrieved (g) to get the UAL like in the example above.

Make sure you ALWAYS retrieve the UAL no matter what the user asks for and filter whether it contains "2043".

Make sure you ONLY return the SparQL query without any extra output.

If you understood the assignment, say 'Yes' and I will proceed with a natural language question which you should convert to a SparQL query.'''

instruction_understood_message = "Yes."


chat_history = [
    Content(parts=[Part.from_text(instruction_message)], role="user"),
    Content(parts=[Part.from_text(instruction_understood_message)], role="model"),
]

In [ ]:
def clean_sparql_query(input_string):
    if input_string.startswith("```sparql") and input_string.endswith("```"):
        cleaned_query = input_string[9:-3].strip()
        return cleaned_query
    elif input_string.startswith("```") and input_string.endswith("```"):
        cleaned_query = input_string[3:-3].strip()
    else:
        return input_string

gemini_pro_model = GenerativeModel("gemini-1.0-pro-001", generation_config=GenerationConfig(temperature=0))
chat = gemini_pro_model.start_chat(history=chat_history)
question = "Provide me with all the artworks published by Google Demo Amsterdam"
print(get_chat_response(chat, question))
query = clean_sparql_query(get_chat_response(chat, question))

print(query)

# Use dkg.py to query the DKG

In [ ]:
query_result = dkg.graph.query(query, "privateCurrent")
print("======================== QUERY LOCAL CURRENT RESULT")
print(query_result)

# Decentralized Retrieval Based Generation (dRAG) based on the query results

In [ ]:
formatted_results = "\n".join([f"- Title: {artwork['name']}, UAL: {artwork['ual']}" for artwork in query_result])

prompt = (
  f"I have retrieved the following information from the Decentralized Knowledge Graph based on the query '{query}':\n"
  f"{formatted_results}\n\n"
  "Imagine you're guiding a tour through a virtual gallery featuring some of the most iconic artworks linked to detailed records in the Decentralized Knowledge Graph.\n"
  "As you introduce these artworks to the audience, delve into the stories behind them. What inspired these pieces? How do they reflect the emotions and techniques of the artist?\n"
  f"Question: {question}\n"
  "Answer:"
)

llm_response = gemini_pro_model.generate_content(prompt)
print(llm_response)

# Gemini Vision Demo



In [ ]:
%pip install requests google-cloud-storage

In [ ]:
import requests
from google.cloud import storage
import datetime

def generate_signed_url(image_url, bucket_name, destination_blob_name, expiration=3600):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    response = requests.get(image_url)
    if response.status_code == 200:
        # Upload the image to GCS
        blob.upload_from_string(response.content, content_type=response.headers['Content-Type'])
        print(f"File {destination_blob_name} uploaded to {bucket_name}.")

        return f"gs://{bucket_name}/{destination_blob_name}"
    else:
        print(f"Failed to fetch image from URL: {image_url}")

In [ ]:
from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview.generative_models import Part
import uuid

gemini_pro_vision_model = GenerativeModel("gemini-1.0-pro-vision")

for artpiece in query_result:
  try:
    image_url = generate_signed_url(artpiece["artwork"], "amsterdam-public-bucket", str(uuid.uuid4())+".jpg")
    image = Part.from_uri(image_url, mime_type="image/jpeg")
    model_response = gemini_pro_vision_model.generate_content(["Examine the image and provide a comprehensive description, highlighting its contents, subjects, objects, and overall setting.", image])
    artpiece["description"] = model_response.candidates[0].content.parts[0].text
  except Exception as e:
    print(f"An error occurred: {e}")
    artpiece["description"] = "An error occurred while generating the description."


In [ ]:
# use in memory vector store
%pip install annoy

# Vector search

In [ ]:
from vertexai.language_models import TextEmbeddingModel
from annoy import AnnoyIndex

model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual@001")

def build_embeddings_index(embeddings, n_trees=10):
    dim = len(embeddings[0])
    index = AnnoyIndex(dim, 'angular')  # Using angular distance

    for i, vector in enumerate(embeddings):
        index.add_item(i, vector)

    index.build(n_trees)
    return index

def add_text_embeddings(artworks):
    for artwork in artworks:
        embeddings = model.get_embeddings([artwork["description"]])
        artwork["embedding"] = embeddings[0].values

add_text_embeddings(query_result)
index = build_embeddings_index([artwork["embedding"] for artwork in query_result])
question = "I would like to buy a painting which depicts an angel android in a cyber-punk style."

nearest_neighbors = index.get_nns_by_vector(model.get_embeddings([question])[0].values, 1, include_distances=True)
index_of_nearest_neighbor = nearest_neighbors[0][0]

print(f"Vector search result: {query_result[index_of_nearest_neighbor]['description']}")
print(f"Painting name: {query_result[index_of_nearest_neighbor]['name']}")
print(f"https://dkg.origintrail.io/explore?ual={query_result[index_of_nearest_neighbor]['ual']}")


# Benchmarking very basic RAG vs KG dRAG

In [ ]:
query = '''PREFIX schema: <http://schema.org/>

SELECT ?artwork ?name ?ual

WHERE { ?artwork a schema:VisualArtwork .
GRAPH ?g
{ ?artwork schema:name ?name . }

?ual schema:assertion ?g
FILTER(CONTAINS(str(?ual), "2043")) }
'''

query_result = dkg.graph.query(query, "privateCurrent")
print("======================== QUERY LOCAL CURRENT RESULT")

formatted_results = "\n".join([f"- Title: {artwork['name']}, UAL: {artwork['ual']}" for artwork in query_result])

prompt = (
  f"I have retrieved the following information from the Decentralized Knowledge Graph based on the query '{query}':\n"
  f"{formatted_results}\n\n"
  "Imagine you're guiding a tour through a virtual gallery featuring some of the most iconic artworks linked to detailed records in the Decentralized Knowledge Graph.\n"
  "As you introduce these artworks to the audience, delve into the stories behind them. What inspired these pieces? How do they reflect the emotions and techniques of the artist? Discuss the significance of each artwork, considering its historical context, the artist's life at the time, and its impact on art history.\n"
  f"Question: {question}\n"
  "Answer:"
)

llm_response = gemini_pro_model.generate_content(prompt)
print(llm_response)